<a href="https://colab.research.google.com/github/mjolnika/nlp/blob/master/4/nlp_4_2_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

НАСТЯ Панасюк

Выберите корпус отзывов: MUSICAL INSTRUMENTS
Допустим, что вам нужно подготовить аналитический отчет по этим отзывам — например, для производителя нового продукта этой категории. Для этого будем искать упоминания товаров в отзывах (будем считать их NE). Учтите, что упоминание может выглядеть не только как "Iphone 10", но и как "модель", "телефон" и т.п.

(3 балла) Предложите 3 способа найти упоминания товаров в отзывах (+)

(2 балла) Реализуйте один из предложенных вами способов. (+)

(1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед) (+)

(3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика ранжирует выше коллокации, подходящие для отчёта).
(-)

(1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров (+)

Бонус (2 балла): если придумаете способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch") (-)

In [13]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Musical_Instruments_5.json.gz')

Муз. инструменты -- довольно ограниченный домен. Сложно представить музыкальный инструмент с каким-то сильно отличающимся названием.

Мысли:


1. Можно взять ворднет и гипонимы из него по ключу musical_instrument и дальше выделять коллокации с кусочками этих гипонимов (bass flute -- смотрим bass, смотрим flute)

2. Или же можно взять эмбеддинги для этих сочетаний и поступить так:
2\1 Получаем эмбеддинг словосочетания. Дальше сравниваем с эмбеддингами токенов из отзыва нужной части речи (это прилагательные и существительные), если подряд идут токены N(A) + N. Также стоит предусмотреть кореференцию для случаев it's amazing. Если в отзыве it's amazing это первое предложение, считать, что это сказано о продукте (т.к. нет никаких кореферентных объектов).

3. Также можно добавить понятия, связанные с музыкой. (Обратиться к тезаурусу, например, вытащить описания из викидаты https://www.wikidata.org/wiki/Q11461 для sound и music). Также было бы здорово добавить меронимы, например, для игрока на скрипке релевантно, чтобы струны были норми и так далее.

Недостатки: много предзаданного, что может не встретиться или встретиться в другом виде, и поэтому такие знания окажутся бесполезными, т.к. люди пишут рецензии не так, как я ожидаю.

4. Вообще, если поступать проще, без такого прекодирования, ограниченного словарем, можно отобрать все существительные из ривью, сравнить их близость с эмбеддингами musical instrument И music, и из топа (количество в топе выбирать по длине текста) можно судить о качестве товара. Sound очень похоже на music, поэтому если sound is awful, то это характеристика товара очень сильно. Дальше можно взять что-то среднее для оценки товара по ривью или просуммировать определения качества. Типа

Sound is good
Nice decoration
Piano keys stick (nb! хорошо бы повыделять именные группы через чанкинг)

Поэтому стоит брать не только прилагательные, но и предикаты в принципе. И тут нужно понять, насколько оценка негативная или позитивная (Keys stick good / Keys awfully stick), так что тут могла бы какая-то оценка тональности пригодиться.

5. Тут еще мог бы быть хорош синтаксический парсинг, чтобы выделять оценку предиката, но тут тоже нужно внедриться в наречия и прилагательные, прописать логику для отрицаний.

Я попробую реализовать хотя бы в базовом виде 4-й вариант.

1) Кусок кода для варианта 1 (может и потом понадобиться, но мне сейчас кажется, что посмотреть эмбеддинги по близости может быть проще по вычислениям)

In [14]:
from collections import defaultdict, Counter
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
from pprint import pprint
import re
mi = wn.synset('musical_instrument.n.01')
hyp = lambda s:s.hyponyms()
for category in mi.tree(hyp):
  print(category)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Synset('musical_instrument.n.01')
[Synset('barrel_organ.n.01')]
[Synset('bass.n.07'), [Synset('bass_fiddle.n.01')], [Synset('bass_guitar.n.01')], [Synset('bass_horn.n.01'), [Synset('euphonium.n.01')], [Synset('helicon.n.01')]], [Synset('bombardon.n.02')]]
[Synset('calliope.n.02')]
[Synset('electronic_instrument.n.01'), [Synset('electric_organ.n.01')], [Synset('synthesizer.n.02')], [Synset('theremin.n.01')]]
[Synset('jew's_harp.n.01')]
[Synset('keyboard_instrument.n.01'), [Synset('accordion.n.01')], [Synset('celesta.n.01')], [Synset('clavichord.n.01')], [Synset('clavier.n.02'), [Synset('harpsichord.n.01'), [Synset('spinet.n.02')], [Synset('virginal.n.01')]]], [Synset('organ.n.05')], [Synset('piano.n.01'), [Synset('grand_piano.n.01'), [Synset('baby_grand.n.01')], [Synset('concert_grand.n.01')]], [Synset('mechanical_piano.n.01')], [Synset('upright.n.02'), [Synset('spinet.n.01

In [15]:
text = '''Synset('musical_instrument.n.01')
[Synset('barrel_organ.n.01')]
[Synset('bass.n.07'), [Synset('bass_fiddle.n.01')], [Synset('bass_guitar.n.01')], [Synset('bass_horn.n.01'), [Synset('euphonium.n.01')], [Synset('helicon.n.01')]], [Synset('bombardon.n.02')]]
[Synset('calliope.n.02')]
[Synset('electronic_instrument.n.01'), [Synset('electric_organ.n.01')], [Synset('synthesizer.n.02')], [Synset('theremin.n.01')]]
[Synset('jew's_harp.n.01')]
[Synset('keyboard_instrument.n.01'), [Synset('accordion.n.01')], [Synset('celesta.n.01')], [Synset('clavichord.n.01')], [Synset('clavier.n.02'), [Synset('harpsichord.n.01'), [Synset('spinet.n.02')], [Synset('virginal.n.01')]]], [Synset('organ.n.05')], [Synset('piano.n.01'), [Synset('grand_piano.n.01'), [Synset('baby_grand.n.01')], [Synset('concert_grand.n.01')]], [Synset('mechanical_piano.n.01')], [Synset('upright.n.02'), [Synset('spinet.n.01')]]], [Synset('synthesizer.n.02')]]
[Synset('music_box.n.01')]
[Synset('percussion_instrument.n.01'), [Synset('bones.n.01')], [Synset('chime.n.01'), [Synset('carillon.n.01')], [Synset('handbell.n.01')]], [Synset('cymbal.n.01'), [Synset('high-hat_cymbal.n.01')], [Synset('zill.n.01')]], [Synset('drum.n.01'), [Synset('bass_drum.n.01')], [Synset('bongo.n.01')], [Synset('snare_drum.n.01')], [Synset('tabor.n.01')], [Synset('tambour.n.02')], [Synset('tambourine.n.01')], [Synset('tenor_drum.n.01')], [Synset('timbrel.n.01')]], [Synset('glockenspiel.n.01')], [Synset('gong.n.01')], [Synset('kettle.n.04')], [Synset('lagerphone.n.01')], [Synset('maraca.n.01')], [Synset('marimba.n.01')], [Synset('piano.n.01'), [Synset('grand_piano.n.01'), [Synset('baby_grand.n.01')], [Synset('concert_grand.n.01')]], [Synset('mechanical_piano.n.01')], [Synset('upright.n.02'), [Synset('spinet.n.01')]]], [Synset('rain_stick.n.01')], [Synset('steel_drum.n.01')], [Synset('triangle.n.05')], [Synset('vibraphone.n.01')]]
[Synset('stringed_instrument.n.01'), [Synset('banjo.n.01')], [Synset('bowed_stringed_instrument.n.01'), [Synset('bass_fiddle.n.01')], [Synset('cello.n.01')], [Synset('viol.n.01'), [Synset('viola_d'amore.n.01')], [Synset('viola_da_braccio.n.01')], [Synset('viola_da_gamba.n.01')]], [Synset('viola.n.03')], [Synset('violin.n.01'), [Synset('amati.n.02')], [Synset('guarnerius.n.03')], [Synset('stradavarius.n.01')]]], [Synset('chordophone.n.01'), [Synset('balalaika.n.01')], [Synset('harp.n.01'), [Synset('aeolian_harp.n.01')], [Synset('lyre.n.01'), [Synset('trigon.n.03')]]], [Synset('lute.n.02')], [Synset('mandolin.n.01'), [Synset('mandola.n.01')]]], [Synset('clavichord.n.01')], [Synset('clavier.n.02'), [Synset('harpsichord.n.01'), [Synset('spinet.n.02')], [Synset('virginal.n.01')]]], [Synset('dulcimer.n.01')], [Synset('guitar.n.01'), [Synset('acoustic_guitar.n.01')], [Synset('bass_guitar.n.01')], [Synset('cittern.n.01')], [Synset('electric_guitar.n.01')], [Synset('hawaiian_guitar.n.01')], [Synset('uke.n.01')]], [Synset('koto.n.01')], [Synset('piano.n.01'), [Synset('grand_piano.n.01'), [Synset('baby_grand.n.01')], [Synset('concert_grand.n.01')]], [Synset('mechanical_piano.n.01')], [Synset('upright.n.02'), [Synset('spinet.n.01')]]], [Synset('psaltery.n.01')], [Synset('samisen.n.01')], [Synset('sitar.n.01')], [Synset('zither.n.01'), [Synset('dulcimer.n.02')]]]
[Synset('wind_instrument.n.01'), [Synset('brass.n.02'), [Synset('baritone.n.03')], [Synset('bass_horn.n.01'), [Synset('euphonium.n.01')], [Synset('helicon.n.01')]], [Synset('bugle.n.01')], [Synset('clarion.n.01')], [Synset('cornet.n.01'), [Synset('serpent.n.03')]], [Synset('flugelhorn.n.01')], [Synset('french_horn.n.01')], [Synset('saxhorn.n.01')], [Synset('trombone.n.01'), [Synset('sackbut.n.01')]]], [Synset('free-reed_instrument.n.01'), [Synset('accordion.n.01')], [Synset('american_organ.n.01')], [Synset('concertina.n.02'), [Synset('bandoneon.n.01')]], [Synset('harmonica.n.01')], [Synset('harmonium.n.01')]], [Synset('kazoo.n.01')], [Synset('ocarina.n.01')], [Synset('organ.n.05')], [Synset('organ_pipe.n.01'), [Synset('flue_pipe.n.01')], [Synset('organ_stop.n.01'), [Synset('diapason.n.01')], [Synset('dulciana.n.01')], [Synset('flue_stop.n.01')], [Synset('reed_stop.n.01'), [Synset('vox_humana.n.01')]], [Synset('sourdine.n.02')], [Synset('vox_angelica.n.01')]], [Synset('reed_pipe.n.01')]], [Synset('pipe.n.04'), [Synset('bagpipe.n.01'), [Synset('musette.n.01')]], [Synset('chanter.n.01')], [Synset('drone.n.05')], [Synset('fipple_flute.n.01'), [Synset('flageolet.n.02'), [Synset('tabor_pipe.n.01')]], [Synset('pennywhistle.n.01')]], [Synset('panpipe.n.01')], [Synset('pitch_pipe.n.01')]], [Synset('post_horn.n.01')], [Synset('whistle.n.03')], [Synset('woodwind.n.01'), [Synset('beating-reed_instrument.n.01'), [Synset('double-reed_instrument.n.01'), [Synset('bassoon.n.01'), [Synset('contrabassoon.n.01')], [Synset('tenoroon.n.01')]], [Synset('english_horn.n.01')], [Synset('krummhorn.n.01')], [Synset('oboe.n.01'), [Synset('heckelphone.n.01')], [Synset('musette_pipe.n.01')], [Synset('oboe_d'amore.n.01')], [Synset('oboe_da_caccia.n.01')], [Synset('shawm.n.01'), [Synset('bombardon.n.02')]]]], [Synset('free-reed.n.01')], [Synset('single-reed_instrument.n.01'), [Synset('clarinet.n.01'), [Synset('b-flat_clarinet.n.01')], [Synset('bass_clarinet.n.01')], [Synset('basset_horn.n.01')]], [Synset('hornpipe.n.03')], [Synset('sax.n.02')]]], [Synset('flute.n.01'), [Synset('fife.n.01')], [Synset('nose_flute.n.01')], [Synset('piccolo.n.01')]]]]'''

terms = re.findall('Synset\(\'([^\.]+).n', text)
terms = [t.replace('_', ' ') for t in terms]
print(terms)

['musical instrument', 'barrel organ', 'bass', 'bass fiddle', 'bass guitar', 'bass horn', 'euphonium', 'helicon', 'bombardon', 'calliope', 'electronic instrument', 'electric organ', 'synthesizer', 'theremin', "jew's harp", 'keyboard instrument', 'accordion', 'celesta', 'clavichord', 'clavier', 'harpsichord', 'spinet', 'virginal', 'organ', 'piano', 'grand piano', 'baby grand', 'concert grand', 'mechanical piano', 'upright', 'spinet', 'synthesizer', 'music box', 'percussion instrument', 'bones', 'chime', 'carillon', 'handbell', 'cymbal', 'high-hat cymbal', 'zill', 'drum', 'bass drum', 'bongo', 'snare drum', 'tabor', 'tambour', 'tambourine', 'tenor drum', 'timbrel', 'glockenspiel', 'gong', 'kettle', 'lagerphone', 'maraca', 'marimba', 'piano', 'grand piano', 'baby grand', 'concert grand', 'mechanical piano', 'upright', 'spinet', 'rain stick', 'steel drum', 'triangle', 'vibraphone', 'stringed instrument', 'banjo', 'bowed stringed instrument', 'bass fiddle', 'cello', 'viol', "viola d'amore",

4) пробую способ 4

In [16]:
from nltk.chunk.util import tree2conlltags,conlltags2tree

In [17]:
nltk.download('averaged_perceptron_tagger')
nltk.download('conll2000')
from nltk.corpus import conll2000
data= conll2000.chunked_sents()
train_data=data[:10900]
test_data=data[10900:]

def conll_tag_chunks(chunk_sents):
    tagged_sents = [tree2conlltags(tree) for tree in chunk_sents]
    return [[(t, c) for (w, t, c) in sent] for sent in tagged_sents]
def combined_tagger(train_data, taggers, backoff=None):
    for tagger in taggers:
        backoff = tagger(train_data, backoff=backoff)
    return backoff

from nltk.tag import UnigramTagger, BigramTagger
from nltk.chunk import ChunkParserI
#Define the chunker class
class NGramTagChunker(ChunkParserI):
  def __init__(self,train_sentences,tagger_classes=[UnigramTagger,BigramTagger]):
    train_sent_tags=conll_tag_chunks(train_sentences)
    self.chunk_tagger=combined_tagger(train_sent_tags,tagger_classes)
  def parse(self,tagged_sentence):
     if not tagged_sentence:
       return None
     pos_tags=[tag for word, tag in tagged_sentence]
     chunk_pos_tags=self.chunk_tagger.tag(pos_tags)
     chunk_tags=[chunk_tag for (pos_tag,chunk_tag) in chunk_pos_tags]
     wpc_tags=[(word,pos_tag,chunk_tag) for ((word,pos_tag),chunk_tag) in zip(tagged_sentence,chunk_tags)]
     return conlltags2tree(wpc_tags)
#train chunker model
ntc=NGramTagChunker(train_data)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


In [64]:
def chunk_sent(sentence):
  nltk_pos_tagged=nltk.pos_tag(sentence.split())
  chunk_tree=ntc.parse(nltk_pos_tagged)
  return chunk_tree

def chunk_text(text):
  sents = re.split('[\.\?!]+', text)
  chunked = []
  for sent in sents:
    chunked.append(chunk_sent(sent))
  return chunked

In [19]:
df.groupby('asin').head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"
...,...,...,...,...,...,...,...,...,...
10253,A2VRAT69JDAD3W,B00JBIVXGC,"Jason Whitt ""Whittmeister""","[0, 0]","I'm a D'Addario man myself, but hey free is fr...",4.0,"I'm a D'Addario man myself, but hey free is ...",1405900800,"07 21, 2014"
10254,A306NASGVUDFKF,B00JBIVXGC,"Jeffrey E ""jeffinaustintx""","[0, 0]",I really like these strings. While they are n...,5.0,I really like these strings,1404432000,"07 4, 2014"
10255,A1TSKKBNV38E8Y,B00JBIVXGC,"K. Harriger ""K.R. Harriger""","[0, 0]",I have lots of friends who play these strings....,3.0,"Hmmm.... I like them, but with a lot of reserv...",1403654400,"06 25, 2014"
10256,A14B2YH83ZXMPP,B00JBIVXGC,Lonnie M. Adams,"[0, 0]","Great, just as expected. Thank to all.",5.0,Five Stars,1405814400,"07 20, 2014"


In [146]:
df.reset_index().groupby('asin').count().reset_index()['index'].sort_values(ascending=False)

706    163
94     143
156    116
707    114
198     93
      ... 
489      5
490      5
493      5
496      5
0        5
Name: index, Length: 900, dtype: int64

Главная задача домашки -- выделить коллокации и описания, а не предикаты, поэтому для начала буду их опускать

Конкатенирую все отзывы на каждый продукт, чтобы посмотреть на похожие чанки и сделать топ. Посмотрим, что выйдет, а дальше подумаем, насколько нужны эмбеддинги.

In [133]:
rev_df = df[['reviewText','asin']].groupby(['asin'])['reviewText'].transform(lambda x: '||'.join(x)).drop_duplicates()

In [134]:
rev_df

0        Not much to write about here, but it does exac...
5        So good that I bought another one.  Love the h...
11       I got it to have it if I needed it. I have fou...
16       This Fender cable is the perfect length for me...
23       Got this cable to run a rockband keyboard cont...
                               ...                        
10199    Notice in the name guitar INPUT. This is not t...
10205    These may be my new favorite bass strings.Like...
10216    Likes:- Pretty smooth.  Minimal fret noise and...
10240    Just put these on my Martin DCX1E and they sou...
10248    I put these strings on my Guild acoustic elect...
Name: reviewText, Length: 900, dtype: object

In [164]:
def np_chunks(chunks):
  np_chunks = []
  for t in chunks:
    if t:
      for i, n in enumerate(t):
          if str(n).startswith('(NP') and re.search('NN', str(n)) and not re.search('[I|i]\'?[a-z]?[a-z]?/NN', str(n)):
            np_chunks.append(str(n))
  return np_chunks

In [166]:
for product in rev_df[3:4]:
  revs = product.split('||')
  all_chunks = []
  for rev in revs:
    chunks = chunk_text(rev)
    #print(chunks)
    npchunks = np_chunks(chunks)
   # print(npchunks)
    all_chunks += npchunks
  #  print(len(all_chunks))
  dict_chunk = Counter(all_chunks)
  pprint(dict_chunk.most_common(10))

[('(NP the/DT cable/NN)', 4),
 ('(NP this/DT cable/NN)', 3),
 ('(NP Sam/NNP Ash/NNP)', 2),
 ('(NP these/DT cables/NNS)', 2),
 ('(NP the/DT connectors/NNS)', 2),
 ('(NP This/DT Fender/NNP cable/NN)', 1),
 ('(NP the/DT perfect/JJ length/NN)', 1),
 ('(NP the/DT metal/NN sleeve/JJ)', 1),
 ('(NP unscrewed/JJ way/NN)', 1),
 ('(NP the/DT color/NN)', 1)]


Даже не зная изначально какой предмет перед нам мы смогли получить его тип (и иногда в топе упоминание компании) через чанкинг. Всякие I've надо почистить, в целом все работает и можно из топа брать существительные.

In [198]:
from string import punctuation
from collections import defaultdict
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english') + [''])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [199]:
maximum = [706, 94, 156, 707, 198] # самое большое количество ривью
list_rev = rev_df.to_list()
for number in maximum:

  print(f'# PRODUCT ID: {number}#\n')
  rev_text = list_rev[number]
  revs = rev_text.split('||')
  all_chunks = []
  for rev in revs:
    chunks = chunk_text(rev)
    #print(chunks)
    npchunks = np_chunks(chunks)
   # print(npchunks)
    all_chunks += npchunks
  #  print(len(all_chunks))
  dict_chunk = Counter(all_chunks)
  products = []
  for key, _ in dict_chunk.most_common(10): # эмпирически 10 норм число которое выделяет основное
    found = re.findall(' ([A-z\']+)/NN', key)
    for n in found:
      products.append(n)

 # pprint(dict_chunk.most_common(10))

  products = set(products)


  tokens = re.split('\.| |\|\|', rev_text.replace('  ', ' '))
  tokens = [token.strip(punctuation) for token in tokens if token.lower() not in stop_words]
  lefts = defaultdict(list)
  rights = defaultdict(list)
  for num, token in enumerate(tokens):
    if token in products:
      if num > 1:
        lefts[token].append(tokens[num-1])
      if num+1 < len(tokens):
        rights[token].append(tokens[num+1])

  for key in products:
    left = Counter(lefts[key])
    right = Counter(rights[key])
    print('\n' + key)
    print('_'*40)
    print('LEFT CONTEXT')
    pprint(left.most_common(10))
    print('RIGHT CONTEXT')
    pprint(right.most_common(10)) # тут так же можно делать сентимент анализ и объединять похожие значения типа awesome cool в одно


  print('\n')


# PRODUCT ID: 706#


Snark
________________________________________
LEFT CONTEXT
[('tuner', 3),
 ('tuners', 2),
 ('recommended', 2),
 ('well', 1),
 ('like', 1),
 ('contacted', 1),
 ('say', 1),
 ('difficult', 1),
 ('meter', 1),
 ('purchase', 1)]
RIGHT CONTEXT
[('SN-1', 6),
 ('tuner', 4),
 ('tuners', 4),
 ('bit', 1),
 ('well', 1),
 ('Marcy', 1),
 ('also', 1),
 ('company', 1),
 ('shows', 1),
 ('music', 1)]

battery
________________________________________
LEFT CONTEXT
[('volt', 2),
 ('time', 1),
 ('bright', 1),
 ('display', 1),
 ('accurate', 1),
 ('long', 1),
 ('guitar', 1),
 ('CHEAP', 1),
 ('opening', 1),
 ('get', 1)]
RIGHT CONTEXT
[('lasts', 2),
 ('life', 2),
 ('compartment', 2),
 ('harness', 1),
 ('dying', 1),
 ('selecting', 1),
 ('seems', 1),
 ('run', 1),
 ('door', 1),
 ('comes', 1)]

Easy
________________________________________
LEFT CONTEXT
[('box', 2),
 ('anyway', 1),
 ('helps', 1),
 ('weight', 1),
 ('life', 1),
 ('money', 1),
 ('knobs', 1),
 ('fantastic', 1),
 ('tuners', 1)]
RIGHT

Прикольно, что в этом смысле даже It's, который чанкер определеяет как НН в тему, так как под ИЦ скрывается объект ривью.

С Easy тоже классно, потому что вышло Easy use, это тоже про аналитику товаров многое говорит. Или battery lasts.

Получается, что через чанкинг мы убили сразу двух зайцев: выделяются не только упоминания продуктов (или их частей, metal sleeve) и их качества, но и отчасти описания, если это нужно. (perfect length), это пригодится для анализа.
Получилось, что даже берт на этом этапе (анализ продуктов) не пригодился, хватает и глупого чанкинга.

Бертом можно уже отделить предметы, и смотреть близость к musical instrument,  после чего смотреть левый-правый контекст.
Но в общем-то и без эмбеддингов все хорошо, поэтому оставляю этот вариант.